In [ ]:
import scipp as sc
import plopp as pp
import numpy as np
from math import sqrt
from scipp.serialization import serialize, deserialize
%matplotlib widget

In [ ]:
coords = np.array([(_x, _y) for _y in np.linspace(-10, 10, 51)
                       for _x in np.linspace(-10, 10, 51)])
x_coords = sc.array(dims=['x'], values=coords[:, 0], unit='m')
y_coords = sc.array(dims=['y'], values=coords[:, 1], unit='m')

In [ ]:
def create_obj():
    return sc.DataArray(data=sc.zeros(sizes={'y': 50, 'x': 50}),
                                 coords={'x': sc.linspace('x', -10, 10, 51, unit='m'),
                                         'y': sc.linspace('y', -10, 10, 51, unit='m')})

da = create_obj()

In [ ]:
da

In [ ]:
fig = pp.figure2d(stream_node)

In [ ]:
fig

In [ ]:
import matplotlib.image as mpimg

In [ ]:
img = mpimg.imread("../../../docs/resources/icon.png")

In [ ]:
indices = [[x, y] for y in range(256) for x in range(256) if sum(img[x][y]) > 0]
# for x in range(256):
#     for y in range(256):
#         if sum(img[x][y]) > 0:
#             indices.append([x, y])

In [ ]:
events = np.array(indices)/256*20-10

In [ ]:
np.random.shuffle(events)

In [ ]:
groups = events[:-(len(events)%32)].reshape((-1,32,2))

In [ ]:
igroup = 0

def update_da():
    global igroup
    igroup += 1
    counts, _, _ = np.histogram2d(groups[igroup][:, 1], -1*groups[igroup][:, 0],
                                  bins=(np.linspace(-10, 10, 51),
                                        np.linspace(-10, 10, 51)))
    da.values += counts
    return da

In [ ]:
stream_node = pp.Node(update_da)

In [ ]:
import asyncio

async def async_range(count):
    for i in range(count):
        yield(i)
        await asyncio.sleep(0.1)

async def my_stream_func(graph):
    async for i in async_range(164):
        graph.notify_children('update')

streaming_task = asyncio.create_task(my_stream_func(stream_node))

In [ ]:
pp.show_graph(stream_node)